In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import re
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

Using TensorFlow backend.


In [2]:
names=['URL','Category']
#df=pd.read_csv( "../input/website-classification-using-url/URL Classification.csv")
#df=pd.read_csv('../input/Website classification using URL/URL Classification.csv')
df=pd.read_csv('../input/website-classification-using-url/URL Classification.csv',names=names, na_filter=False)

In [3]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(df['Category'])
df['Category'] = lb.transform(df['Category'])

data = pd.get_dummies(df,prefix=['Category'], columns = ['Category'])
df = data
df[:2]

,URL,Category_0,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,Category_10,Category_11,Category_12,Category_13,Category_14
1,http://www.liquidgeneration.com/,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,http://www.onlineanime.org/,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df1 = df[1:4000]
df2 = df[50000:54000]
df3 = df[520000:524000]
df4 =df[535300:539300]
df5 = df[650000:654000]
df6= df[710000:714000]
df7=  df[764200:768200]
df8=  df[793080:797080]
df9=  df[839730:843730]
df10=  df[850000:854000]
df11=  df[955250:959250]
df12=  df[1013000:1017000]
df13=  df[1143000:1147000]
df14=  df[1293000:1297000]
df15=  df[1492000:1496000]
#df6 = df[77000:1562978]
dt=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15], axis=0)
df.drop(df.index[1:4000],inplace= True)
df.drop(df.index[50000:54000],inplace= True)
df.drop(df.index[520000:524000],inplace= True)
df.drop(df.index[535300:539300],inplace= True)
df.drop(df.index[650000:654000],inplace= True)
df.drop(df.index[710000:714000],inplace= True)
df.drop(df.index[764200:768200],inplace= True)
df.drop(df.index[793080:797080],inplace= True)
df.drop(df.index[839730:843730],inplace= True)
df.drop(df.index[850000:854000],inplace= True)
df.drop(df.index[955250:959250],inplace= True)
df.drop(df.index[1013000:1017000],inplace= True)
df.drop(df.index[1143000:1147000],inplace= True)
df.drop(df.index[1293000:1297000],inplace= True)
df.drop(df.index[1492000:1496000],inplace= True)
df.tail()

,URL,Category_0,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,Category_10,Category_11,Category_12,Category_13,Category_14
1562974,http://www.maxpreps.com/,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562975,http://www.myscore.com/,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562976,http://sportsillustrated.cnn.com/highschool,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562977,http://rss.cnn.com/rss/si_highschool?format=xml,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562978,http://www.usatoday.com/sports/preps/,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
X_train=df['URL']
y_train=df.iloc[: , 1:16].values
print(y_train)
y_train.shape

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


(1502979, 15)

In [6]:
X_test=dt['URL']
y_test=dt.iloc[: , 1:16].values
print(y_test)
y_test.shape

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


(59999, 15)

import re
import nltk
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(2,2))

gs_clf = Pipeline([('vect', stemmed_count_vect),
                   ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='perceptron', penalty='l2',
                    alpha =1e-4 , max_iter=20 ,tol=None)),
   ])
gs_clf = gs_clf.fit(X_train, y_train)


**This is for understandind the basics of stemming it transforms the words to its root**
* >  stem_vectorizer = stemmed_count_vect
* > text1 = 'ï am so much bored because of your meaningless behaviour'
* > print(stem_vectorizer.fit_transform([text1]))
* > print(stem_vectorizer.get_feature_names())

In [7]:
from keras.preprocessing.text import Tokenizer
def create_and_train_tokenizer(texts):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(texts)
    return tokenizer

from keras.preprocessing.sequence import pad_sequences
def encode_reviews(tokenizer, max_length, docs):
    encoded=tokenizer.texts_to_sequences(docs) 
    padded=pad_sequences(encoded, maxlen=max_length, padding="post")
    return padded

tokenizer=create_and_train_tokenizer(texts = X_train)
vocab_size=len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

max_length=max([len(row.split()) for row in X_train])
print("Maximum length:",max_length)

X_train_encoded = encode_reviews(tokenizer, max_length, X_train)
X_test_encoded = encode_reviews(tokenizer, max_length, X_test)
print('x_train shape:', X_train_encoded.shape)
print('x_test shape:', X_test_encoded.shape)

Vocabulary size: 1182241
Maximum length: 16
x_train shape: (1502979, 16)
x_test shape: (59999, 16)


In [8]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras import layers, models
from keras.callbacks import EarlyStopping

In [9]:
X_train_encoded[:2]

array([[     1,      2, 201614,      3,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0],
       [     1,      2,   3662,   3193,      3,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0]],
      dtype=int32)

In [10]:
from keras import layers, models

def create_embedding_model(vocab_size, max_length):
    model=models.Sequential()
    model.add(layers.Embedding(vocab_size, 100, input_length=max_length))

    model.add(layers.Conv1D(1024, 5, activation="relu"))
    #model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D())

    model.add(layers.Conv1D(1024, 5, activation="relu"))
    #model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D())

    model.add(layers.Flatten())
    model.add(layers.Dense(512,  activation="relu")) 
    dropout = Dropout(0.5)
    model.add(layers.Dense(15,  activation="softmax"))   
    return model

embedding_model = create_embedding_model(vocab_size=vocab_size, max_length=max_length)
embedding_model.summary()

from keras.optimizers import SGD
#opt = SGD(lr=0.01, momentum=0.9)
embedding_model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16, 100)           118224100 
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12, 1024)          513024    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 1024)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 1024)           5243904   
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 1024)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

In [11]:

#earlyStopping = EarlyStopping(monitor="val_accuracy", patience=1)
modelHistory = embedding_model.fit(X_train_encoded, 
                                   y_train, 
                                   validation_data=(X_test_encoded, y_test),
                                   epochs= 15
                                   )

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1502979 samples, validate on 59999 samples
Epoch 1/15
1502979/1502979 [==============================] - 2061s 1ms/step - loss: 1.6712 - accuracy: 0.4813 - val_loss: 1.9090 - val_accuracy: 0.4441
Epoch 2/15
1502979/1502979 [==============================] - 2054s 1ms/step - loss: 0.8913 - accuracy: 0.7315 - val_loss: 1.0954 - val_accuracy: 0.7217
Epoch 3/15
1502979/1502979 [==============================] - 2053s 1ms/step - loss: 0.3888 - accuracy: 0.8884 - val_loss: 0.7224 - val_accuracy: 0.8035
Epoch 4/15
1502979/1502979 [==============================] - 2064s 1ms/step - loss: 0.2848 - accuracy: 0.9174 - val_loss: 0.6280 - val_accuracy: 0.8308
Epoch 5/15
1502979/1502979 [==============================] - 2065s 1ms/step - loss: 0.2385 - accuracy: 0.9283 - val_loss: 0.6470 - val_accuracy: 0.8441
Epoch 6/15
1502979/1502979 [==============================] - 2054s 1ms/step - loss: 0.2149 - accuracy: 0.9349 - val_loss: 0.6253 - val_accuracy: 0.8485
Epoch 7/15
1502979/1502979 [==

In [12]:
print(len(y_train) + len(y_test))
print(len(X_train_encoded))

1562978
1502979


In [13]:
_, acc = embedding_model.evaluate(X_train_encoded, y_train, verbose=0)
print("Train accuracy:{:.2f}".format(acc*100))
_,acc= embedding_model.evaluate(X_test_encoded, y_test, verbose=0)
print("Test accuracy:{:.2f}".format(acc*100))

Train accuracy:96.47
Test accuracy:85.66


print(gs_clf.predict(['http://www.banglainfotube.com']))
print(gs_clf.predict(['http://www.gamespot.net/']))

In [19]:
import tensorflow as tf
#saving the model
embedding_model.save('classification_model.h5')
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('classification_model.h5')
# Show the model architecture
new_model.summary()
# Evaluate the model
loss, acc = new_model.evaluate(X_test_encoded, y_test, verbose=0)
print('Restored model, accuracy 1: {:5.2f}%'.format(100*acc))

''''''''''
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')
# Create a new model instance
model = create_model()
# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')
# Evaluate the model
loss,acc = model.evaluate(X_test_encoded, y_test, verbose=0)
print("Restored model, accuracy 2: {:5.2f}%".format(100*acc))''''



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16, 100)           118224100 
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12, 1024)          513024    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 1024)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 1024)           5243904   
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 1024)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

NameError: name 'model' is not defined

In [15]:
test_maxlength = 0
for text in X_test:
    length = len(text) 
    if test_maxlength < length:
        test_maxlength = length
        ntext = text
        
print('maxlength is {} and text is = {} '.format(test_maxlength , ntext))

maxlength is 312 and text is = http://www.ams.usda.gov/amsv1.0/ams.fetchtemplatedata.do?template=templaten&amp;navid=soybeanprogrambackgroundinformation&amp;rightnav1=soybeanprogrambackgroundinformation&amp;topnav=&amp;leftnav=industrymarketingandpromotion&amp;page=soybeanbackgroundofsoybeancheckoffprogram&amp;resulttype=&amp;acct=lspromores 
